In [0]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
import tensorflow as tf
from tensorflow.python.framework import ops
import pandas as pd
%matplotlib inline
np.random.seed(1)
import scipy.io

In [0]:
import cv2

In [0]:
import tensorflow_hub as hub

In [0]:
train_x = pd.read_csv("/content/drive/My Drive/flower_recog/train.csv")

In [0]:
valid_x = pd.read_csv("/content/drive/My Drive/flower_recog/valid.csv")

In [0]:
train_x.set_index('Unnamed: 0',drop = True,inplace=True)
valid_x.set_index('Unnamed: 0',drop = True,inplace=True)

In [0]:
X = train_x.iloc[:,:-1].values
Y = train_x['Labels'].values

In [0]:
X = X.reshape(len(train_x),128,128,3)
X1 = valid_x.iloc[:,:-1].values
Y1 = valid_x['Labels'].values
X1 = X1.reshape(len(valid_x),128,128,3)
X = X/255
X1 = X1/255
from sklearn.preprocessing import OneHotEncoder
Y_trans = OneHotEncoder().fit_transform(Y.reshape(-1,1)).toarray()
Y1_trans = OneHotEncoder().fit_transform(Y1.reshape(-1,1)).toarray()

In [0]:
image = []
for i in range(1020):
  image.append(cv2.resize(X[i,:,:,:],(299,299)))

In [0]:
X = np.array(image)

In [0]:
image = []
for i in range(1020):
  image.append(cv2.resize(X1[i,:,:,:],(299,299)))
X1 = np.array(image)
X1.shape

(1020, 299, 299, 3)

In [0]:
#from sklearn.model_selection import train_test_split

In [0]:
#X_train ,X_test ,Y_train, Y_test = train_test_split(X,Y_trans,test_size = 0.2, random_state = 10)

In [0]:
#X_train = X_train/255
#X_test = X_test/255

In [0]:
#X_train = X_train.reshape(len(X_train),150,150,3)
#X_test = X_test.reshape(len(X_test),150,150,3)

In [0]:
def batches(X_train,Y_train):
    X_train_set = []
    Y_train_set = []
    for i in range(len(X_train)//128):
        X_train_set.append(X_train[i*128:(i+1)*128,:,:,:])
        Y_train_set.append(Y_train[i*128:(i+1)*128,:])
    return X_train_set,Y_train_set    

In [0]:
def create_placeholders(n_H0, n_W0, n_C0, n_y):
    X = tf.placeholder(tf.float32, shape=(None, n_H0, n_W0, n_C0))
    Y = tf.placeholder(tf.float32, shape=(None, n_y))
    return X, Y

In [0]:
"""def initialize_parameters():
    
    tf.set_random_seed(1)                              #
    W1 = tf.get_variable("W1", [3,3,3,64], initializer =tf.contrib.layers.xavier_initializer(seed = 0))
    W2 =tf.get_variable("W2", [3,3,64,128], initializer =tf.contrib.layers.xavier_initializer(seed = 0))
    W3 =tf.get_variable("W3", [3,3,128,128], initializer =tf.contrib.layers.xavier_initializer(seed = 0))
    W4 =tf.get_variable("W4", [3,3,128,256], initializer =tf.contrib.layers.xavier_initializer(seed = 0))
    parameters = {"W1": W1,
                  "W2": W2,
                  "W3": W3,
                  "W4": W4}
    
    return parameters""""

In [0]:
def forward_propagation(X):
    module = hub.Module("https://tfhub.dev/google/imagenet/inception_v3/feature_vector/1")
    outputs = module(X)
    Z5 = tf.layers.dense(inputs=outputs, units=102)
    return Z5

In [0]:
def compute_cost(Z5, Y):
    """
    Computes the cost
    
    Arguments:
    Z3 -- output of forward propagation (output of the last LINEAR unit), of shape (6, number of examples)
    Y -- "true" labels vector placeholder, same shape as Z3
    
    Returns:
    cost - Tensor of the cost function
    """
    
    ### START CODE HERE ### (1 line of code)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = Z5, labels = Y))
    ### END CODE HERE ###
    
    return cost

In [0]:
def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.001,
          num_epochs = 100, print_cost = True):
    """
    Implements a three-layer ConvNet in Tensorflow:
    CONV2D -> RELU -> MAXPOOL -> CONV2D -> RELU -> MAXPOOL -> FLATTEN -> FULLYCONNECTED
    
    Arguments:
    X_train -- training set, of shape (None, 64, 64, 3)
    Y_train -- test set, of shape (None, n_y = 6)
    X_test -- training set, of shape (None, 64, 64, 3)
    Y_test -- test set, of shape (None, n_y = 6)
    learning_rate -- learning rate of the optimization
    num_epochs -- number of epochs of the optimization loop
    minibatch_size -- size of a minibatch
    print_cost -- True to print the cost every 100 epochs
    
    Returns:
    train_accuracy -- real number, accuracy on the train set (X_train)
    test_accuracy -- real number, testing accuracy on the test set (X_test)
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    
    ops.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables
    tf.set_random_seed(1)                             # to keep results consistent (tensorflow seed)
    seed = 3                                          # to keep results consistent (numpy seed)
    (m, n_H0, n_W0, n_C0) = X_train.shape             
    n_y = Y_train.shape[1]                            
    costs = []                                        # To keep track of the cost
    
    # Create Placeholders of the correct shape
    ### START CODE HERE ### (1 line)
    X, Y = create_placeholders(n_H0, n_W0, n_C0, n_y)
    ### END CODE HERE ###
    ###parameters = initialize_parameters()
    # Initialize parameters
    ### START CODE HERE ### (1 line)
    ### END CODE HERE ###
    
    # Forward propagation: Build the forward propagation in the tensorflow graph
    ### START CODE HERE ### (1 line)
    Z5 = forward_propagation(X)
    ### END CODE HERE ###
    
    # Cost function: Add cost function to tensorflow graph
    ### START CODE HERE ### (1 line)
    cost = compute_cost(Z5, Y)
    ### END CODE HERE ###
    
    # Backpropagation: Define the tensorflow optimizer. Use an AdamOptimizer that minimizes the cost.
    ### START CODE HERE ### (1 line)
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
    ### END CODE HERE ###
    
    # Initialize all the variables globally
    init = tf.global_variables_initializer()
     
    # Start the session to compute the tensorflow graph
    with tf.Session() as sess:
        
        # Run the initialization
        sess.run(init)
        
        # Do the training loop
        for epoch in range(num_epochs):

            minibatch_cost = 0
            X_set , Y_set = batches(X_train, Y_train)

            for x,y in zip(X_set,Y_set):

                # Select a minibatch
                minibatch_X, minibatch_Y = x,y
                # IMPORTANT: The line that runs the graph on a minibatch.
                # Run the session to execute the optimizer and the cost, the feedict should contain a minibatch for (X,Y).
                _ , temp_cost = sess.run([optimizer, cost], feed_dict={X: minibatch_X, Y: minibatch_Y})
  
                
                minibatch_cost += temp_cost / len(X_set)
                

            # Print the cost every epoch
            if print_cost == True and epoch % 5 == 0:
                print ("Cost after epoch %i: %f" % (epoch, minibatch_cost))
            if print_cost == True and epoch % 1 == 0:
                costs.append(minibatch_cost)
        
        
        # plot the cost
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per tens)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()

        # Calculate the correct predictions
        predict_op = tf.argmax(Z5, 1)
        correct_prediction = tf.equal(predict_op, tf.argmax(Y, 1))
        
        # Calculate accuracy on the test set
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        print(accuracy)
        train_accuracy = accuracy.eval({X: X_train, Y: Y_train})
        test_accuracy = accuracy.eval({X: X_test, Y: Y_test})
        print("Train Accuracy:", train_accuracy)
        print("Test Accuracy:", test_accuracy)
                
        return train_accuracy, test_accuracy

In [0]:
_, _= model(X, Y_trans, X1, Y1_trans)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
Use keras.layers.Dense instead.


Instructions for updating:
Use keras.layers.Dense instead.


Instructions for updating:
Please use `layer.__call__` method instead.


Instructions for updating:
Please use `layer.__call__` method instead.


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Cost after epoch 0: 4.564566
Cost after epoch 5: 1.836981
